# ComeBackHome with provide current posistion

In [8]:
from numba import jit, njit
import random
import numpy as np

In [9]:
@njit 
def getAgentSize():
    return 10
@njit
def getStateSize():
    return 5
@njit   
def getActionSize():
    return 4

In [3]:
@njit 
def getReward(p_state):
    if p_state[0] == p_state[1] and p_state[2] == p_state[3]:
        return 1
    else:
        return -1
@njit 
def checkEnded(e_state):
    size = 10
    for i in range(0,10):
        if  e_state[size*9*size+i*2] == e_state[size*9*size+20] and e_state[size*9*size+i*2+1] == e_state[size*9*size+21] :
            return i
    return -1
    

In [4]:
@njit   
def initEnv():
    n = 1
    p = 0.5
    # the size of the matrix 
    size = 10
    grid = np.random.binomial(n,p, size=(size,size))
    first_row = grid[0]
    first_row[first_row == 1] = 0
    grid[0] = first_row
    for i in range(1,size):
        grid[i,size-1] = 1
    # create environment
    # from 0 to 899: map 
    # from 900 to 919: player position
    # from 920 to 921: end position
    # from 922 to 923: end position
    # 924: turn

    env = np.full(size*9*size+23,0)

    # create the map 
    map = np.full((size*3, size*3),1)
    i = 0
    j = 0
    while i < size:
        w = i*3 + 1
        while j < size:
            k = j*3 + 1
            toss = grid[i,j]
            map[w,k] = 0 # ' '
            if toss == 0 and k+2 < size*3:
                map[w,k+1] = 0 # ' '
                map[w,k+2] = 0 #' '
            if toss == 1 and w-2 >=0:
                map[w-1,k] = 0 #' '
                map[w-2,k] = 0 #' '
            j = j + 1
        i = i + 1
        j = 0

    # create end and start point
    rows, cols = (np.where(map == 0))


    # start point
    start = random.randint(0, len(rows)-1)
    # print('choice start x y ',rows[start],cols[start])
    
    # end point 
    end = random.randint(0, len(rows)-1)
    if end == start :
        end = random.randint(0, len(rows)-1)
    # print('choice end x y ',rows[end],cols[end])

    
    map = map.reshape(1,size*9*size)[0]
    # print(map)
    env[np.where(map== 1)[0]] = 1
    env[np.where(map== 0)[0]] = 0

    # current_agent_posistion_player
    for i in range(0,10):
        env[size*9*size+i*2]= rows[start]
        env[size*9*size+i*2+1]= cols[start]

    # end position
    env[size*9*size+20] = rows[end]
    env[size*9*size+21] = cols[end]

    # Turn
    env[size*9*size+22] = 0

    return env

@njit 
def getAgentState(e_state):
    size = 10
    pIdx = e_state[size*9*size+22] % 10 # player index

    p_state = np.full(5,0) 

    # The position of the agent
    p_state[0] = e_state[size*9*size+pIdx*2] 
    p_state[1] = e_state[size*9*size+pIdx*2+1]

    # The position of the end
    p_state[2] = e_state[size*9*size+20]
    p_state[3] = e_state[size*9*size+21] 

    p_state[4] = pIdx # player position
    # The posistion around the player
    

    return p_state

@njit
def stepEnv(action,env):
    size = 10
    #player index
    pIdx = env[size*9*size+22] % 10
    # current position
    cur_x = env[size*9*size+pIdx*2]
    cur_y = env[size*9*size+pIdx*2+1]

    # go left
    if action == 0:
        if cur_y - 1 >= 0 and env[(cur_x)*size*3+cur_y-1] == 0:
            env[size*9*size+pIdx*2] = cur_x 
            env[size*9*size+pIdx*2+1] = cur_y-1

    # go straight
    if action == 1:
        if cur_x - 1 >= 0 and env[(cur_x-1)*size*3+cur_y] == 0:
            env[size*9*size+pIdx*2] = cur_x-1
            env[size*9*size+pIdx*2+1] = cur_y 

    # go down
    if action == 2:
        if cur_x + 1 < size*3 and env[(1+cur_x)*size*3+cur_y] == 0:
            env[size*9*size+pIdx*2] = cur_x+1
            env[size*9*size+pIdx*2+1] = cur_y 

    # go right
    if action == 3:
        if cur_y + 1 < size*3 and env[(cur_x)*size*3+cur_y+1] == 0:
            env[size*9*size+pIdx*2] = cur_x 
            env[size*9*size+pIdx*2+1] = cur_y+1

    env[size*9*size+22]+=1  # Turn
@njit 
def getValidActions(p_state):
    if 
    validActions = np.full(4, 1)
    return validActions

# Visualize bàn chơi

In [ ]:
def player_random(p_state, temp_file, per_file2):
    arr_action = getValidActions(p_state)
    arr_action = np.where(arr_action == 1)[0]
    act_idx = np.random.randint(0, len(arr_action))
    return arr_action[act_idx], temp_file, per_file2


from os import environ
import numpy as np
from numba.typed import List

def run(listAgent, perData):
    size = 10
    env = initEnv()
    tempData = []
    for _ in range(10):
        dataOnePlayer = List()
        dataOnePlayer.append(np.array([[0.]]))
        tempData.append(dataOnePlayer)

    winner = -1
    print('Start State')
    for i in range(0,10):
        print(f'P{i+1}:',end=" ")
        print(env[size*9*size+i*2],env[size*9*size+i*2+1],end=" ")
    print(' ')
    print(' ')
    
    output = env[0:size*9*size].reshape(size*3,size*3)

    print(output)

    output_grid1 = np.empty([size*3, size*3],dtype=str)
    output_grid1[:] = '#'
    for i in range(len(output)):
        for j in range(len(output)):
            if output[i,j] == 0:
                output_grid1[i,j] = ' '
    for elm in output_grid1:
        print(" ".join(elm))
    
    while env[size*9*size+22]<4000:
        pIdx = env[size*9*size+22]  % 10
        # for i in range(0,10):
        #     print(f'P{i+1}:',end=" ")
        #     print(env[size*9*size+i*2],env[size*9*size+i*2+1],end=" ")
        # print(' ')
        # print(f'Turn {env[size*9*size+22]+1}',end=" ")
        action, tempData[pIdx], perData = listAgent[pIdx](getAgentState(env), tempData[pIdx], perData)
        if action == 0:  #go straight   
            print(f'Player: {pIdx+1} đi trái')
        if action == 1 : # go left :
            print(f'Player: {pIdx+1}  đi thẳng')
        if action == 2 : # go right :
            print(f'Player: {pIdx+1}  đi lùi')
        if action == 3 : # go back :
            print(f'Player: {pIdx+1}  đi phải')
        stepEnv(action, env)
        winner = checkEnded(env)
        
        if winner != -1:
            for i in range(0,10):
                print(f'P{i+1}:',end=" ")
                print('X',env[size*9*size+i*2],env[size*9*size+i*2+1],end=" ")
            print(' ')
            print('Player winner : Player ',winner+1)
            print('số turn',env[size*9*size+22]+1)
            break
    
    for pIdx in range(10):
        env[size*9*size+22] = pIdx
        action, tempData[pIdx], perData = listAgent[pIdx](getAgentState(env), tempData[pIdx], perData)
    
    return winner, perData


def main(listAgent, times, perData, printMode=False, k=100):
    if len(listAgent) != 10:
        raise Exception('Hệ thống chỉ cho phép có đúng 10 người chơi!!!')
    
    numWin = np.full(11, 0)
    pIdOrder = np.arange(10)
    for _ in range(times):
        if printMode and _ != 0 and _ % k == 0:
            print(_, numWin)

        np.random.shuffle(pIdOrder)
        shuffledListAgent = [listAgent[i] for i in pIdOrder]
        winner, perData = run(shuffledListAgent, perData)
        # print('w',winner)
        print('real player ',pIdOrder[winner])

        if winner == -1:
            numWin[10] += 1
        else:
            numWin[pIdOrder[winner]] += 1

    if printMode:
        print(_+1, numWin)

    return numWin, perData
list_player = [player_random,player_random,player_random,player_random,player_random,player_random,player_random,player_random,player_random,player_random]
per = []
count,per1 = main(list_player,1,per)

# Numba run 

In [11]:
from numba.typed import List

@njit
def numbaRun(p0, perData, pIdOrder):
    env= initEnv()
    tempData = []
    for _ in range(1):
        dataOnePlayer = List()
        dataOnePlayer.append(np.array([[0.]]))
        tempData.append(dataOnePlayer)
    size = 10
    winner = -1
    while True :
        pIdx = env[size*9*size+22] % 10
        if pIdOrder[pIdx] == 0:
            action, tempData[pIdx], perData = p0(getAgentState(env), tempData[pIdx], perData)

        stepEnv(action, env)
        winner = checkEnded(env)
        if winner != -1:
            # print('Player winner : Player ',winner+1)
            # print('số turn',env[size*9*size+22]+1)
            break
    
    for pIdx in range(1):
        env[size*9*size+22] = pIdx
        if pIdOrder[pIdx] == 0:
            action, tempData[pIdx], perData = p0(getAgentState(env), tempData[pIdx], perData)
            
    return winner, perData

@njit
def numbaMain(p0, times, perData, printMode=False, k=100):
    numWin = np.full(2, 0)
    pIdOrder = np.arange(1)
    for _ in range(times):
        if printMode and _ != 0 and _ % k == 0:
            print(_, numWin)

        np.random.shuffle(pIdOrder)
        winner, perData = numbaRun(p0, perData, pIdOrder)

        if winner == -1:
            numWin[1] += 1
        else:
            numWin[pIdOrder[winner]] += 1
    
    if printMode:
        print(_+1, numWin)

    return numWin, perData
@njit
def numbaRandomBot(state, tempData, perData):
    validActions = getValidActions(state)
    validActions = np.where(validActions==1)[0]
    idx = np.random.randint(0, len(validActions))
    return validActions[idx], tempData, perData

perx = [np.array([np.random.rand(getActionSize()) for _ in range(100)]),np.zeros((1,100)),np.zeros((1,100)),np.zeros((1,100))]
count,per1 = numbaMain(numbaRandomBot,1000,perx)
print(count)

/var/folders/ds/4lcv95w11y7brm6dqhc04s240000gn/T/ipykernel_2755/2172304898.py:41: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'perData' of function 'numbaRun'.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "../../../../var/folders/ds/4lcv95w11y7brm6dqhc04s240000gn/T/ipykernel_2755/2172304898.py", line 3:
<source missing, REPL/exec in use?>

  winner, perData = numbaRun(p0, perData, pIdOrder)
/var/folders/ds/4lcv95w11y7brm6dqhc04s240000gn/T/ipykernel_2755/2172304898.py:41: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'perData' of function 'numbaRandomBot'.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

Fi

IndexError: getitem out of range